In [ ]:
using LinearAlgebra     

# Compute the largest eigenvalue and its corresponding eigenvector of a real square matrix `A` using the power method.
""" 
# Arguments
- `A`: A square matrix with only real entries.
- `num_iterations`: Maximum number of iterations (default: 1000).
- `tolerance`: Convergence threshold for eigenvalue change (default: 1e-6).
- `round_digits`: Number of decimal places to round the results (default: 4).

# Returns
A tuple `(eigenvalue, eigenvector)` where:
- `eigenvalue`: The leading eigenvalue of `A`.
- `eigenvector`: The corresponding normalized eigenvector.
"""
function power_method_eigen(A::AbstractMatrix{T}; num_iterations::Int=1000, tolerance::Float64=1e-8, round_digits::Int=4) where T<:Real
    # Ensure that A is square
    n, m = size(A)
    if n != m
        error("Matrix A must be square.")
    end

    # Handle the special case for zero matrix
    # We start with a random vector to approximate the leading eigenvector without prior knowledge about the matrix
    if all(A .== 0)
        # Return the first standard basis vector
        leading_eigenvector = zeros(m)
        leading_eigenvector[1] = 1.0
        return (0, leading_eigenvector, 0.0)  # Eigenvalue for a zero matrix is always 0
    end

    # Initialize vector 
    eigenvector = rand(m)
    eigenvector /= norm(eigenvector)

    
    # Normalize the vector
    # Normalization prevents numerical overflow or underflow during iterations and ensures stability in the computation
    eigenvector /= norm(eigenvector)

    eigenval_old = 0.0
    eigenval = 0.0

    for i in 1:num_iterations

        # Compute new vector 
        eigenvector_new = A * eigenvector
        
        # Compute the Rayleigh quotient as an estimate of the eigenvalue 
        if (!(dot(eigenvector, eigenvector) == 0))
            eigenval = dot(eigenvector_new, eigenvector) / dot(eigenvector, eigenvector) 
        end 

        # Normalize new vector 
        # Normalization prevents numerical overflow or underflow during iterations and ensures stability in the computation
        if (!(norm(eigenvector_new) == 0))
            eigenvector_new /= norm(eigenvector_new) 
        end
        

        # If old eigenvalue approx = new eigenvalue, terminate
        # The difference between consecutive eigenvalue estimates is used as convergence criteria
        if abs(eigenval - eigenval_old) < tolerance
            break
        end 

        # Update new eigenvector approximation
        eigenvector = eigenvector_new
        # Set old reference singular value to current for next iteration for the termination check  
        eigenval_old = eigenval
    end 

    # Rounding is applied to improve the readability of the results and avoid floating-point precision issues in the output
    eigenval = round(eigenval, digits=round_digits)
    eigenvector = round.(eigenvector, digits=round_digits)

    return eigenval, eigenvector
end

# Compute the largest singular value and its corresponding singular vectors of a matrix `A` using the power method.
"""
# Arguments
- `A`: A matrix with only real entries.
- `num_iterations`: Maximum number of iterations to perform (default: 1000).
- `tolerance`: Convergence threshold for singular value change (default: 1e-6).
- `round_digits`: Number of decimal places to round the results (default: 4).

# Returns
A tuple `(singular_value, left_singular_vector, right_singular_vector)` where:
- `singular_value`: The largest singular value of `A`.
- `left_singular_vector`: The corresponding left singular vector of `A`.
- `right_singular_vector`: The corresponding right singular vector of `A`. 
"""
function power_method_singular(A::AbstractMatrix{T}; num_iterations::Int=1000, tolerance::Float64=1e-6, round_digits::Int=4) where T<:Real
    n, m = size(A)
    # Initialize a random vector
    # We start with a random vector to approximate the singular vectors of the largest singular value without prior knowledge about the matrix
    x = rand(m)
    # Normalize the vector
    # Normalization prevents numerical overflow or underflow during iterations and ensures stability in the computation
    x /= norm(x)
 

    # Handle the special case for zero matrix
    if all(A .== 0)
        # Return the first standard basis vector
        leading_eigenvector = zeros(m)
        leading_eigenvector[1] = 1.0
        return (0, leading_eigenvector, leading_eigenvector, 0.0)  # Eigenvalue for a zero matrix is always 0
    end

    singularval_old = 0.0
    singularval = 0.0

    # Calculate A^T A 
    # The matrix A^T A is used because its eigenvalues correspond to the squares of the singular values of A
    A_transposed = transpose(A)
    A_transposed_times_A = A_transposed * A;   

    # Initialize left and right singular vectors  
    u = zeros(m)   
    v = zeros(m)  
    
    for i in 1:num_iterations
        # Multiply A^TA by x to compute the new vector
        x_new = A_transposed_times_A * x 

        # Normalize x to get the new right singular vector approximation
        # Normalization prevents numerical overflow or underflow during iterations and ensures stability in the computation
        v = x_new / norm(x_new)

        # Compute corresponding singular value approximation
        # The singular value is the norm of A multiplied by the current approximation of the right singular vector
        singularval = norm(A * v)

        # Compute left singular vector approximation
        u = A * v / singularval
 
        # If old singular value approx = new singular value, terminate
        # The difference between consecutive eigenvalue estimates is used as convergence criteria
        if abs(singularval - singularval_old) < tolerance
            break
        end

        # Normalize x for the next iteration
        # Normalization prevents numerical overflow or underflow during iterations and ensures stability in the computation
        x = x_new / norm(x_new)
        # Set old reference singular value to current for next iteration for the termination check 
        singularval_old = singularval
    end

    # Rounding is applied to improve the readability of the results and avoid floating-point precision issues in the output
    singularval = round(singularval, digits=round_digits)
    u = round.(u, digits=round_digits)
    v = round.(v, digits=round_digits)
    
    return singularval, u, v
end


# Testing 

A = [10 1 2; 0 1 1; 0 0 1
]
eigenval, v = power_method_eigen(A)
println("Leading eigenvalue: ", eigenval)
println("Leading eigenvector: ", v)

println();
# Compute eigenvalues and eigenvectors
eigen_result = eigen(A)

# Find the index of the largest eigenvalue
largest_index = argmax(eigen_result.values)

# Print the largest eigenvalue
println("Largest eigenvalue: ", eigen_result.values[largest_index])

# Print the corresponding eigenvector
println("Corresponding eigenvector:")
println(eigen_result.vectors[:, largest_index])


println();
println();

singularval, u, v = power_method_singular(A)
println("Leading singular value: ", singularval)
println("Leading left singular vector: ", u)
println("Leading right singular vector: ", v)

println();
# Perform SVD
svd_result = svd(A)

# Print the first singular value
println("First singular value: ", svd_result.S[1])

# Print the first left singular vector (U matrix, first column)
println("First left singular vector (U[:,1]):")
println(svd_result.U[:, 1])

# Print the first left singular vector (U matrix, first column)
println("First right singular vector (V[:,1]):")
println(svd_result.V[:, 1])


Leading eigenvalue: 10.0
Leading eigenvector: [1.0, 0.0, 0.0]

Largest eigenvalue: 10.0
Corresponding eigenvector:
[1.0, 0.0, 0.0]


Leading singular value: 10.2531
Leading left singular vector: [0.9994, 0.0293, 0.0195]
Leading right singular vector: [0.9747, 0.1003, 0.1997]

First singular value: 10.253136846644898
First left singular vector (U[:,1]):
[0.9993820308927207, 0.029261279730430988, 0.01947649448125296]
First right singular vector (V[:,1]):
[0.974708565622769, 0.10032474217485468, 0.19969516320921077]


In [ ]:

 
# A = [6 2 -45 ;
# -2 2 1 ;
# 23 23 0]

# A = [6 2;
# -2 2 ;
# 23 23]


# singularval, u, v = power_method_singular(A)
# println("Leading singular value: ", singularval)
# println("Leading left singular vector: ", u)
# println("Leading right singular vector: ", v)
 

# println("\n \n \n");

# A = [10 1 2 ; 0 1 1; 0 0  1]


# eigenval, v = power_method_eigen(A)
# println("Leading eigenvalue: ", eigenval)
# println("Leading eigenvector: ", v)


# singularval, u, v = power_method_singular(A)
# println("Leading singular value: ", singularval)
# println("Leading left singular vector: ", u)
# println("Leading right singular vector: ", v)

# A = [0 0 0 ; 0 0 0; 0 0 0]


# eigenval, v = power_method_eigen(A)
# println("Leading eigenvalue: ", eigenval)
# println("Leading eigenvector: ", v)
# # println("count: ", counter)


# singularval, u, v = power_method_singular(A)
# println("Leading singular value: ", singularval)
# println("Leading left singular vector: ", u)
# println("Leading right singular vector: ", v)
# # println("count: ", counter)


# A = [2 -1 0 0 0 ; -1 2 -1 0 0 ; 0 -1 2 -1 0; 0 0 -1 2 -1 ; 0 0 0 -1 2]


# eigenval, v = power_method_eigen(A)
# println("Leading eigenvalue: ", eigenval)
# println("Leading eigenvector: ", v)
 

# singularval, u, v = power_method_singular(A)
# println("Leading singular value: ", singularval)
# println("Leading left singular vector: ", u)
# println("Leading right singular vector: ", v*1.73490631506)
 


# # Perform SVD
# svd_result = svd(A)

# # Print the singular values
# println("Singular values: ", svd_result.S)

# # Print the left singular vectors (U matrix)
# println("Left singular vectors (U):")
# println(svd_result.U)

# # Print the right singular vectors (V matrix)
# println("Right singular vectors (V):")
# println(svd_result.V)


# Leading eigenvalue: 15.0
# Leading eigenvector: [0.6667, -0.6667, 0.3333]
# Leading singular value: 15.0
# Leading left singular vector: [0.6667, -0.6667, 0.3333]
# Leading right singular vector: [0.6667, -0.6667, 0.3333]
# Leading singular value: 33.0188
# Leading left singular vector: [0.1726, -0.0013, 0.985]
# Leading right singular vector: [0.7176, 0.6965]

 
 

# Leading eigenvalue: 10.0
# Leading eigenvector: [1.0, 0.0, 0.0]
# Leading singular value: 10.2531
# Leading left singular vector: [0.9994, 0.0293, 0.0195]
# Leading right singular vector: [0.9747, 0.1003, 0.1997]
# Leading eigenvalue: 0
# Leading eigenvector: [1.0, 0.0, 0.0]
# Leading singular value: 0
# Leading left singular vector: [1.0, 0.0, 0.0]
# Leading right singular vector: [1.0, 0.0, 0.0]
# Leading eigenvalue: 3.7321
# Leading eigenvector: [0.2887, -0.5, 0.5773, -0.5, 0.2887]
# Leading singular value: 3.7321
# Leading left singular vector: [0.2883, -0.4996, 0.5774, -0.5004, 0.2891]
# Leading right singular vector: [0.500000000000292, -0.86658570437247, 1.001561415684138, -0.8683206106875299, 0.501734906315352]

